## Prep

In [2]:
import pandas as pd
import numpy as np
import os
import tqdm
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from itertools import product
from math import floor
import time

import tensorflow as tf
import tensorflow_federated as tff
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.callbacks import CSVLogger

# -> check tff
#print(tff.federated_computation(lambda: 'Hello World')()) 

2023-05-08 12:04:51.142963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 12:04:51.834632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
df_locs = [
    '../output/data/insurance-clean.csv',
    "https://raw.githubusercontent.com/Olhaau/fl-official-statistics-addon/main/output/data/insurance-clean.csv"
]

def load_df(df_locs):
    """ Loads data from a path to a csv-file.
    
    :param df_locs: possible locations of a CSV file
    :type df_locs: str or list of str
    :output: Ingested Data.
    :rtype: pandas.DataFrame 
    """
    df = pd.DataFrame()

    if isinstance(df_locs, str): df_locs = [df_locs]
    
    for df_loc in df_locs:
        try:
            df = pd.read_csv(df_loc, index_col = 0)
            print("loaded data from {}".format(df_loc))
            if len(df) != 0: break
        except Exception as ex:
            print("{} in ".format(type(ex).__name__), df_loc)

    return df

df = load_df(df_locs)
df.head()

loaded data from ../output/data/insurance-clean.csv


,age,sex,bmi,children,smoker,region,charges,region0,region1,region2,region3
0,0.021739,0.0,0.321227,0.0,1.0,southwest,16884.92400,0.0,0.0,0.0,1.0
1,0.000000,1.0,0.479150,0.2,0.0,southeast,1725.55230,0.0,0.0,1.0,0.0
2,0.217391,1.0,0.458434,0.6,0.0,southeast,4449.46200,0.0,0.0,1.0,0.0
3,0.326087,1.0,0.181464,0.0,0.0,northwest,21984.47061,0.0,1.0,0.0,0.0
4,0.304348,1.0,0.347592,0.0,0.0,northwest,3866.85520,0.0,1.0,0.0,0.0


In [16]:
features = ['age', 'sex', 'bmi', 'children', 'smoker'
            , 'region0', 'region1', 'region2', 'region3']
target = 'charges'

In [28]:

nclients = 4
region_testsize = 20

df_in = df.loc[:, features + [target]]

dfs_fed_rand = []

for _ in range(nclients):

    df_sample = df_in.sample(frac = 1. / nclients).reset_index()

    X_test  = df_sample.loc[0:region_testsize, features[:9]]
    y_test  = df_sample.loc[0:region_testsize, target  ]
    X_train = df_sample.loc[(region_testsize + 1): , features[:9]]
    y_train = df_sample.loc[(region_testsize + 1): , target  ]

    # -> possiblely a huge overlap, not distinct

    fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))


    train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
    test_set = (X_test, y_test)

    dfs_fed_rand.append((train_set, test_set))

dfs_fed_rand

[(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (         age  sex       bmi  children  smoker  region0  region1  region2  \
   0   0.000000  0.0  0.385930       0.0     0.0      1.0      0.0      0.0   
   1   0.456522  1.0  0.440678       0.4     0.0      0.0      0.0      1.0   
   2   0.739130  1.0  0.488028       0.0     0.0      0.0      0.0      1.0   
   3   0.000000  0.0  0.529056       0.0     0.0      1.0      0.0      0.0   
   4   0.021739  1.0  0.103309       0.0     0.0      0.0      0.0      0.0   
   5   0.760870  0.0  0.288943       0.4     0.0      0.0      0.0      0.0   
   6   0.956522  0.0  0.595507       0.4     0.0      1.0      0.0      0.0   
   7   0.260870  0.0  0.621200       0.6     1.0      0.0      0.0      1.0   
   8   0.891304  1.0  0.255582       0.2     0.0      1.0      0.0      0.0   
   9   0.673913  0.0  0.212133       0.6     1.0      1.0   

In [ ]:
# Create test and train sets and put them into random_client_ds, use four clients which are independent of the region
def get_dataset_random_region(dataset, num_clients=4, test_size_per_region=20):
    """Creates a list with client datasets independent of the region.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param num_clients: the number of clients create (equal big datasets per client), default value is 4 clients
    :type num_clients: int, optional
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: List of the prepared dataset with one entry per region, the test values and labels for each region
    :rtype: List of (tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series)"""
    size_of_client_ds = int(dataset.shape[0] / num_clients)

    dataset_to_split = dataset.copy()
    dataset_to_split.pop("region")
    random_client_ds = []
    for i in range(num_clients):
        sampled = dataset_to_split.sample(n=size_of_client_ds)
        dataset_to_split.drop(sampled.index)

        X_test = sampled.head(test_size_per_region)
        y_test = X_test.pop('charges')

        X_train = sampled.tail(size_of_client_ds - test_size_per_region)
        y_train = X_train.pop('charges')

        fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

        train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
        test_set = (X_test, y_test)

        random_client_ds.append((train_set, test_set))

    return random_client_ds

In [7]:
NUM_CLIENTS = 4
NUM_EPOCHS = 50
BATCH_SIZE = 128
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5
NUM_ROUNDS = 50
RUN_NAME = f'0,8-3({NUM_ROUNDS})-{NUM_EPOCHS}-epochs-{BATCH_SIZE}-batch-WithRegion/'

In [17]:
def build_model(
    nfeatures = 9,
    units = [40, 40, 20], 
    activations = ['relu'] * 3, 
    compile = True,
    loss = 'mean_squared_error',
    optimizer = tf.optimizers.legacy.Adam(learning_rate = .05),
    metrics = ["mae", 'mean_squared_error', r2_score], 
    run_eagerly = True
    ):
  """Construct a fully connected neural network and compile it.
  
  Parameters
  ------------
  nfeatures: int, optional
    Number of input features. Default is 9.
  units: list of int, optional
    List of number of units of the hidden dense layers. The length of ``units`` defines the number of hidden layers. Default are 3 layers with 40, 40 an 20 units, respectively.
  activations: list of str, optional
    List of activation functions used in the hidden layers.
  loss: str, optional
    Used loss function for compiling.
  optimizer: keras.optimizers, optional
    Used optimizer for compiling.
  metrics: list of str or sklearn.metrics
    List of metrics for compiling.
  run_eagerly: bool
    Parameter for compiling

  Return
  ------------
    model: keras.engine.sequential.Sequential
      Keras sequential fully connected neural network. Already compiled.
  """
  
  # construct model
  model = Sequential()
  model.add(InputLayer(input_shape = [nfeatures]))
  for ind in range(len(units)):
    model.add(Dense(
      units = units[ind], 
      activation = activations[ind]
      ))
  model.add(Dense(1))
  
  # compile model
  if compile:
    model.compile(
      loss = loss,
      optimizer = optimizer,
      metrics = metrics,
      run_eagerly = run_eagerly
    )

  return model

In [24]:
def build_model_fed(nfeatures = 9):
    _model = build_model(nfeatures = nfeatures, compile = False)

    return tff.learning.models.from_keras_model(
        _model,
        input_spec = (
        tf.TensorSpec((None, nfeatures), dtype = tf.float64),
        tf.TensorSpec((None,), dtype = tf.float64)),
        loss = tf.keras.losses.MeanAbsoluteError(),
        #loss = tf.keras.losses.MeanSquaredError(),
        metrics = [tf.keras.metrics.MeanAbsoluteError()
        #metrics = [tf.keras.losses.MeanSquaredError()
        #,tfa.metrics.RSquare()
        ]
    )
    

In [25]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    build_model_fed,
    client_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),
    server_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),)

In [26]:
print(iterative_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,40],
      float32[40],
      float32[40,40],
      float32[40],
      float32[40,20],
      float32[20],
      float32[20,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[9,40],
    float32[9,40],
    float32[40],
    float32[40],
    float32[40,40],
    float32[40,40],
    float32[40],
    float32[40],
    float32[40,20],
    float32[40,20],
    float32[20],
    float32[20],
    float32[20,1],
    float32[20,1],
    float32[1],
    float32[1]
  >
>@SERVER)


In [27]:
state = iterative_process.initialize()

2023-05-08 15:07:15.647807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:07:15.648291: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:07:15.653357: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:07:15.655720: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:07:15.656157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:07:15.656242: I tensor

In [41]:
# Train the federated model with random clients
for round_num in tqdm.tqdm(range(1, NUM_ROUNDS)):
    result = iterative_process.next(state, [f[0] for f in dfs_fed_rand])
    state = result.state
    metrics = result.metrics
    print(dict(result.metrics['client_work']['train'].items()))
    for name, value in metrics['client_work']['train'].items():
        tf.summary.scalar(name, value, step=round_num)

  4%|▍         | 2/49 [00:00<00:08,  5.32it/s]

{'mean_absolute_error': 1657.0, 'loss': 1652.4829, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1678.5884, 'loss': 1673.9203, 'num_examples': 62600, 'num_batches': 492}


  8%|▊         | 4/49 [00:00<00:08,  5.26it/s]

{'mean_absolute_error': 1654.7894, 'loss': 1650.375, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1656.7637, 'loss': 1652.449, 'num_examples': 62600, 'num_batches': 492}


 12%|█▏        | 6/49 [00:01<00:08,  5.18it/s]

...

 96%|█████████▌| 47/49 [00:08<00:00,  5.16it/s]

{'mean_absolute_error': 1626.6943, 'loss': 1622.7867, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1619.9398, 'loss': 1615.758, 'num_examples': 62600, 'num_batches': 492}


 98%|█████████▊| 48/49 [00:09<00:00,  5.01it/s]

{'mean_absolute_error': 1623.0593, 'loss': 1619.0189, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1634.1056, 'loss': 1629.4027, 'num_examples': 62600, 'num_batches': 492}


100%|██████████| 49/49 [00:09<00:00,  5.25it/s]


In [42]:
metrics['client_work']

OrderedDict([('train',
              OrderedDict([('mean_absolute_error', 1634.1056),
                           ('loss', 1629.4027),
                           ('num_examples', 62600),
                           ('num_batches', 492)]))])

In [37]:
metrics

OrderedDict([('distributor', ()),
             ('client_work',
              OrderedDict([('train',
                            OrderedDict([('mean_absolute_error', 1662.4181),
                                         ('loss', 1658.4181),
                                         ('num_examples', 62600),
                                         ('num_batches', 492)]))])),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', OrderedDict([('update_non_finite', 0)]))])

In [38]:
# Create the test data for model evaluation
X_test = pd.concat([f[1][0] for f in random_client_ds])
y_test = pd.concat([f[1][1] for f in random_client_ds])

test_sets = [tf.data.Dataset.from_tensor_slices(
    (tf.convert_to_tensor(np.expand_dims(el[1][0], axis=0)), 
    tf.convert_to_tensor(np.expand_dims(el[1][1], axis=0)))) 
    for el in dfs_fed_rand]

In [40]:
# Model evaluation
evaluation = tff.learning.build_federated_evaluation(build_model_fed)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
train_metrics = evaluation(model_weights, test_sets)
train_metrics

/tmp/ipykernel_30136/1504518853.py:2: DeprecationWarning: `tff.learning.build_federated_evaluation` is deprecated, use `tff.learning.algorithms.build_fed_eval` instead.
  evaluation = tff.learning.build_federated_evaluation(build_model_fed)
2023-05-08 15:12:18.020792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:12:18.020866: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:12:18.020983: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:12:18.021504: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:12:18.021576: I ten

OrderedDict([('eval',
              OrderedDict([('mean_absolute_error', 2113.0056),
                           ('loss', 2113.0056),
                           ('num_examples', 84),
                           ('num_batches', 4)]))])

LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[array([[-2.10389066e+00,  2.46973348e+00, -3.98962587e-01,
        -2.87362516e-01, -6.82170093e-02,  4.29228187e-01,
         3.13949537e+00,  3.00360703e+00,  1.61716866e+00,
        -1.88885316e-01, -2.36831918e-01, -3.92093331e-01,
         1.12893891e+00,  2.24689770e+00,  3.02883768e+00,
         2.23611426e+00,  1.80612624e+00,  1.47674477e+00,
         3.08594298e+00,  9.85755622e-01,  4.73701566e-01,
         2.95895934e+00, -1.73218712e-01,  9.75096226e-01,
         1.25473809e+00, -1.36465088e-01, -8.98067374e-04,
         8.31643760e-01,  2.96576190e+00,  9.36854184e-01,
        -5.10068893e-01, -7.59894788e-01,  2.98085928e-01,
         2.63044548e+00,  3.05045104e+00, -1.01940072e+00,
         2.26928020e+00, -2.39246383e-01, -7.04018652e-01,
        -3.51507378e+00],
       [-2.89265800e+00,  2.44060591e-01, -1.12642121e+00,
         1.13991871e-01,  2.04161391e-01, -1.15195775e+00,
        -3.89408231e-

## ...

In [48]:
def model_fed(nfeatures = 5):
    def _model():
        return build_model_fed(nfeatures = nfeatures)

    iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
        _model,
        client_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),
        server_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),)
    return iterative_process

In [49]:
def train_model_fed(model, dats_fed, epochs = 50):
    state = model.initialize()
    hist = []
    states = []

    for round_num in range(1, epochs):
        result = model.next(state, [f[0] for f in dats_fed])
        state = result.state
        metrics = dict(result.metrics['client_work']['train'].items())

        # used for tensorboard
        #for name, value in metrics['client_work']['train'].items():
        #    tf.summary.scalar(name, value, step=round_num)
        states.append(state)
        hist.append(metrics)

        print('round  {:2d}, metrics={}'.format(round_num, metrics))

    return model, hist, states, metrics

In [51]:
model = model_fed(9)
hist = train_model_fed(model, dfs_fed_rand, epochs = 100)

2023-05-08 15:22:05.878282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:22:05.878343: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:22:05.878467: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:22:05.879024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:22:05.879171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:22:05.879242: I tensor

round   1, metrics={'mean_absolute_error': 5167.9233, 'loss': 5177.4077, 'num_examples': 62600, 'num_batches': 492}
round   2, metrics={'mean_absolute_error': 4970.0474, 'loss': 4978.626, 'num_examples': 62600, 'num_batches': 492}
round   3, metrics={'mean_absolute_error': 4840.868, 'loss': 4848.5737, 'num_examples': 62600, 'num_batches': 492}
round   4, metrics={'mean_absolute_error': 4714.678, 'loss': 4721.738, 'num_examples': 62600, 'num_batches': 492}
round   5, metrics={'mean_absolute_error': 4620.2354, 'loss': 4626.7163, 'num_examples': 62600, 'num_batches': 492}
round   6, metrics={'mean_absolute_error': 4474.3545, 'loss': 4480.14, 'num_examples': 62600, 'num_batches': 492}
round   7, metrics={'mean_absolute_error': 4360.3706, 'loss': 4365.549, 'num_examples': 62600, 'num_batches': 492}
round   8, metrics={'mean_absolute_error': 4279.247, 'loss': 4284.246, 'num_examples': 62600, 'num_batches': 492}
round   9, metrics={'mean_absolute_error': 4166.1484, 'loss': 4170.274, 'num_exam

In [52]:
# Model evaluation
evaluation = tff.learning.build_federated_evaluation(build_model_fed)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(hist[2][0])
train_metrics = evaluation(model_weights, test_sets)
train_metrics

/tmp/ipykernel_30136/102693261.py:2: DeprecationWarning: `tff.learning.build_federated_evaluation` is deprecated, use `tff.learning.algorithms.build_fed_eval` instead.
  evaluation = tff.learning.build_federated_evaluation(build_model_fed)
2023-05-08 15:24:02.335857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:24:02.335926: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:24:02.336068: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:24:02.336619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:24:02.336709: I tens

OrderedDict([('eval',
              OrderedDict([('mean_absolute_error', 12344.756),
                           ('loss', 12344.755),
                           ('num_examples', 84),
                           ('num_batches', 4)]))])

## Minimal example

In [54]:
# ingest data

df_locs = [
    '../output/data/insurance-clean.csv',
    "https://raw.githubusercontent.com/Olhaau/fl-official-statistics-addon/main/output/data/insurance-clean.csv"
]

def load_df(df_locs):
    """ Loads data from a path to a csv-file.
    
    :param df_locs: possible locations of a CSV file
    :type df_locs: str or list of str
    :output: Ingested Data.
    :rtype: pandas.DataFrame 
    """
    df = pd.DataFrame()

    if isinstance(df_locs, str): df_locs = [df_locs]
    
    for df_loc in df_locs:
        try:
            df = pd.read_csv(df_loc, index_col = 0)
            print("loaded data from {}".format(df_loc))
            if len(df) != 0: break
        except Exception as ex:
            print("{} in ".format(type(ex).__name__), df_loc)

    return df

df = load_df(df_locs)

features = ['age', 'sex', 'bmi', 'children', 'smoker'
            , 'region0', 'region1', 'region2', 'region3']
target = 'charges'

loaded data from ../output/data/insurance-clean.csv


In [55]:
# create data shards

NUM_CLIENTS = 4
NUM_EPOCHS = 50
BATCH_SIZE = 128
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5
NUM_ROUNDS = 50
RUN_NAME = f'0,8-3({NUM_ROUNDS})-{NUM_EPOCHS}-epochs-{BATCH_SIZE}-batch-WithRegion/'

nclients = 4
region_testsize = 20

df_in = df.loc[:, features + [target]]

dfs_fed_rand = []

for _ in range(nclients):

    df_sample = df_in.sample(frac = 1. / nclients).reset_index()

    X_test  = df_sample.loc[0:region_testsize, features[:9]]
    y_test  = df_sample.loc[0:region_testsize, target  ]
    X_train = df_sample.loc[(region_testsize + 1): , features[:9]]
    y_train = df_sample.loc[(region_testsize + 1): , target  ]

    # -> possiblely a huge overlap, not distinct

    fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))


    train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
    test_set = (X_test, y_test)

    dfs_fed_rand.append((train_set, test_set))

dfs_fed_rand

[(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (         age  sex       bmi  children  smoker  region0  region1  region2  \
   0   0.673913  1.0  0.414044       0.2     0.0      1.0      0.0      0.0   
   1   0.413043  1.0  0.822437       0.6     0.0      0.0      0.0      1.0   
   2   0.434783  1.0  0.319478       0.4     0.0      0.0      1.0      0.0   
   3   0.000000  1.0  0.378531       0.2     0.0      0.0      0.0      1.0   
   4   0.130435  0.0  0.375706       0.0     0.0      0.0      1.0      0.0   
   5   0.000000  1.0  0.150794       0.0     1.0      1.0      0.0      0.0   
   6   0.195652  0.0  0.236481       0.0     1.0      0.0      0.0      1.0   
   7   1.000000  1.0  0.598063       0.0     0.0      1.0      0.0      0.0   
   8   0.000000  1.0  0.189131       0.0     0.0      1.0      0.0      0.0   
   9   0.304348  1.0  0.348937       0.2     1.0      0.0   

In [56]:
def build_model(nfeatures = 9,
    units = [40, 40, 20], 
    activations = ['relu'] * 3, 
    compile = True,
    loss = 'mean_squared_error',
    optimizer = tf.optimizers.legacy.Adam(learning_rate = .05),
    metrics = ["mae", 'mean_squared_error', r2_score], 
    run_eagerly = True
    ):
  """Construct a fully connected neural network and compile it.
  
  Parameters
  ------------
  nfeatures: int, optional
    Number of input features. Default is 9.
  units: list of int, optional
    List of number of units of the hidden dense layers. The length of ``units`` defines the number of hidden layers. Default are 3 layers with 40, 40 an 20 units, respectively.
  activations: list of str, optional
    List of activation functions used in the hidden layers.
  loss: str, optional
    Used loss function for compiling.
  optimizer: keras.optimizers, optional
    Used optimizer for compiling.
  metrics: list of str or sklearn.metrics
    List of metrics for compiling.
  run_eagerly: bool
    Parameter for compiling

  Return
  ------------
    model: keras.engine.sequential.Sequential
      Keras sequential fully connected neural network. Already compiled.
  """
  
  # construct model
  model = Sequential()
  model.add(InputLayer(input_shape = [nfeatures]))
  for ind in range(len(units)):
    model.add(Dense(
      units = units[ind], 
      activation = activations[ind]
      ))
  model.add(Dense(1))
  
  # compile model
  if compile:
    model.compile(
      loss = loss,
      optimizer = optimizer,
      metrics = metrics,
      run_eagerly = run_eagerly
    )

  return model

In [57]:
def build_model_fed(nfeatures = 9):
    _model = build_model(nfeatures = nfeatures, compile = False)

    return tff.learning.models.from_keras_model(
        _model,
        input_spec = (
        tf.TensorSpec((None, nfeatures), dtype = tf.float64),
        tf.TensorSpec((None,), dtype = tf.float64)),
        loss = tf.keras.losses.MeanAbsoluteError(),
        #loss = tf.keras.losses.MeanSquaredError(),
        metrics = [tf.keras.metrics.MeanAbsoluteError()
        #metrics = [tf.keras.losses.MeanSquaredError()
        #,tfa.metrics.RSquare()
        ]
    )
    

In [67]:
# Train FL

iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    build_model_fed,
    client_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),
    server_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),)

state = iterative_process.initialize()

for round_num in tqdm.tqdm(range(1, 100)):
    result = iterative_process.next(state, [f[0] for f in dfs_fed_rand])
    state = result.state
    metrics = result.metrics
    print(dict(result.metrics['client_work']['train'].items()))
   # for name, value in metrics['client_work']['train'].items():
   #     tf.summary.scalar(name, value, step=round_num)

2023-05-08 15:34:50.477234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:34:50.477297: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:34:50.477409: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:34:50.477787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:34:50.477865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:34:50.477923: I tensor

{'mean_absolute_error': 5237.533, 'loss': 5247.949, 'num_examples': 62600, 'num_batches': 492}


  2%|▏         | 2/99 [00:01<01:12,  1.33it/s]

{'mean_absolute_error': 5004.08, 'loss': 5012.9453, 'num_examples': 62600, 'num_batches': 492}


  4%|▍         | 4/99 [00:02<00:36,  2.63it/s]

...

{'mean_absolute_error': 1912.7805, 'loss': 1910.3322, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1909.7723, 'loss': 1907.2172, 'num_examples': 62600, 'num_batches': 492}


 98%|█████████▊| 97/99 [00:20<00:00,  5.04it/s]

{'mean_absolute_error': 1901.1832, 'loss': 1898.5841, 'num_examples': 62600, 'num_batches': 492}


100%|██████████| 99/99 [00:21<00:00,  4.71it/s]

{'mean_absolute_error': 1901.1166, 'loss': 1898.6069, 'num_examples': 62600, 'num_batches': 492}
{'mean_absolute_error': 1910.9197, 'loss': 1908.4355, 'num_examples': 62600, 'num_batches': 492}


In [69]:
# Eval FL - Calc
evaluation = tff.learning.build_federated_evaluation(build_model_fed)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
#model_weights = iterative_process.get_model_weights(hist[2][0])

perf = evaluation(model_weights, test_sets)

/tmp/ipykernel_30136/4064712942.py:2: DeprecationWarning: `tff.learning.build_federated_evaluation` is deprecated, use `tff.learning.algorithms.build_fed_eval` instead.
  evaluation = tff.learning.build_federated_evaluation(build_model_fed)
2023-05-08 15:39:26.215722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:39:26.215810: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-08 15:39:26.215932: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-08 15:39:26.216425: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:39:26.216550: I ten

In [70]:
# Eval FL - Output
train_metrics

OrderedDict([('eval',
              OrderedDict([('mean_absolute_error', 12344.756),
                           ('loss', 12344.755),
                           ('num_examples', 84),
                           ('num_batches', 4)]))])